In [108]:
import requests,time,os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from mako.template import Template

In [ ]:
fetchGuancha("WangXiangSui",2)

In [118]:
def fetchGuancha(author,n):
    
    # create directory
    dirname = './'+author+'/img'
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        print("Directory " , dirname ,  " Created ")
    
    # fetch the links and titles of all posts
    link = []
    url = "https://www.guancha.cn/"
    for i in np.arange(1,n+1):
        response = requests.get(url+author+'/list_'+str(i)+'.shtml')
        doc = BeautifulSoup(response.content, features="lxml")
        for j in doc.findAll(class_="module-title"):
            id = j('a')[0].attrs['href'].replace(".shtml","")
            title = j('a')[0].text
            if author in id:
                link.append((id,title))
        time.sleep(1)
    
    # generate the index webpage
    INDEX = Template("""
    <!DOCTYPE html><html><head><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name=viewport><meta charset=utf-8>
    <link rel="stylesheet" href="init.css">
    <style>
    .LI li{
    list-style-type: disc;
    }
    .LI li{
    margin-bottom:8px;
    }
    .LI a{
    text-decoration: none;
    }
    .LI .title{
    color:#000;
    }
    .LI .title:hover{
    color:#f40;
    }
    .LI .date{
    font-size:12px;
    color:#999;
    float:right;
    }
    </style>
    </head>
    <body><div class="BODY">
    <h1>${author}的博客</h1>
    <ul class="LI">
    %for url, name in li:
    <li><a class="title" href="./${url}.html">${name}</a></li>
    %endfor
    </ul>
    </div></body></html>
    """)
    with open(author+".html","w") as index:
        index.write(INDEX.render(author=author,li=link))
    
    # fetch webpages from links
    for path,title in link:
        response = requests.get(url+path+'_s.shtml')
        doc = BeautifulSoup(response.content, features="lxml")
        date = doc.find(class_="time fix")('span')[0].text
        post = doc.find(class_="content all-txt")
        img_id = path.replace('/'+author+'/','')
        count = 1
        for i in post('img'):
            img_url = i.attrs['src']
            img = requests.get(img_url).content
            imgfile = "/img/"+img_id+"_"+str(count)
            with open("./%s%s" % (author, imgfile), "wb") as f:
                f.write(img)
            i.attrs['src'] = "."+imgfile
            count = count+1
    
        # save webpage data as html
        HTML = Template("""<!DOCTYPE html><html><head><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name=viewport><meta charset=utf-8>
        <link rel="stylesheet" href="../init.css">
        <title>${title}</title>
        </head>
        <body><div class="BODY">
        <div class="BACK"><a href="../${author}.html">返回索引页</a></div>
        <h1>${title}</h1>
        <p class="DATE">${date}</p>
        <div class="POST">${post}</div>
        <div class="BACK"><a href="../${author}.html">返回索引页</a></div>
        </body></html>""")
        with open('./'+author+'/'+img_id+'.html','w') as f:
            f.write(HTML.render(author=author,title=title,date=date,post=post))
        time.sleep(2)
    
    return